<a href="https://colab.research.google.com/github/Raymond-Iacobacci/Parallelized-NN/blob/main/MLPPCLSTMNNNLPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load and format datasets 

In [1]:
import numpy as np
import tensorflow as tf
#from python.keras import backend as k
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Conv1D
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)
!pip install tf-nightly

In [2]:
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [3]:
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

# Making model 1 (LSTM)

In [4]:
embedding_vecor_length = 32
model_5 = Sequential()
model_5.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model_5.add(LSTM(50))
model_5.add(Dense(1, activation = 'sigmoid'))
model_5.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model_5.summary())
model_5.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model_5.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 50)                16600     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 176,651
Trainable params: 176,651
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 97s 245ms/step - loss: 0.5515 - accuracy: 0.6904
Epoch 2/3
391/391 [==============================] - 95s 243ms/step - loss: 0.2715 - accuracy: 0.8940
Epoch 3/3
391/391 [==============================] - 96s 245ms/step - loss: 0.2092 - accuracy: 0.9206
Accuracy: 87.72%


In [5]:
from keras import backend as K
model_LSTM = K.function([model_5.layers[0].input],
                                  [model_5.layers[2].output])
layer_output = model_LSTM([X_test])[0]

In [ ]:
print(model_LSTM([X_test])[0][0])

[0.07295863]


# Making model 2 (C1D)

In [6]:
embedding_vecor_length = 32
model_3 = Sequential()
model_3.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model_3.add(Conv1D(1,4))
model_3.add(Flatten())
model_3.add(Dense(1, activation = 'sigmoid'))
model_3.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model_3.summary())
model_3.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model_3.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d (Conv1D)              (None, 497, 1)            129       
_________________________________________________________________
flatten (Flatten)            (None, 497)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 498       
Total params: 160,627
Trainable params: 160,627
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 17s 42ms/step - loss: 0.6097 - accuracy: 0.6282
Epoch 2/3
391/391 [==============================] - 16s 42ms/step - loss: 0.2452 - accuracy: 0.9022
Epoch 3/3
391/391 [====================

In [7]:
from keras import backend as K
model_Conv1D = K.function([model_3.layers[0].input],
                         [model_3.layers[3].output])
layer_output = model_Conv1D([X_test])[0]

In [ ]:
print(model_Conv1D([X_test])[0][0])

[0.28339362]


# Making model 3 (MTP)

In [8]:
embedding_vecor_length = 32
model_4 = Sequential()
model_4.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model_4.add(Dense(100, activation = 'relu'))
model_4.add(Dense(10, activation = 'relu'))
model_4.add(Flatten())
model_4.add(Dense(1, activation = 'sigmoid'))
model_4.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model_4.summary())
model_4.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model_4.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dense_2 (Dense)              (None, 500, 100)          3300      
_________________________________________________________________
dense_3 (Dense)              (None, 500, 10)           1010      
_________________________________________________________________
flatten_1 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5001      
Total params: 169,311
Trainable params: 169,311
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 22s 54ms/step - loss: 0.6016 - accuracy: 0.6132
Epoch 2/

In [9]:
from keras import backend as K
model_MTP = K.function([model_4.layers[0].input],
                         [model_4.layers[4].output])
layer_output = model_MTP([X_test])[0]

# Making composite model (MTP)

In [10]:
print(X_train.shape)
final_train = np.zeros(shape = (25000, 3))
LSTM_list = model_LSTM([X_train])[0]
Conv1D_list = model_Conv1D([X_train])[0]
MTP_list = model_MTP([X_train])[0]
print("Lists copied")
for i in range(25000):
  final_train[i][0] = LSTM_list[i]
  final_train[i][1] = Conv1D_list[i]
  final_train[i][2] = MTP_list[i]
  if i%500 == 0:
    print(i,"/",25000)

(25000, 500)
Lists copied
0 / 25000
500 / 25000
1000 / 25000
1500 / 25000
2000 / 25000
2500 / 25000
3000 / 25000
3500 / 25000
4000 / 25000
4500 / 25000
5000 / 25000
5500 / 25000
6000 / 25000
6500 / 25000
7000 / 25000
7500 / 25000
8000 / 25000
8500 / 25000
9000 / 25000
9500 / 25000
10000 / 25000
10500 / 25000
11000 / 25000
11500 / 25000
12000 / 25000
12500 / 25000
13000 / 25000
13500 / 25000
14000 / 25000
14500 / 25000
15000 / 25000
15500 / 25000
16000 / 25000
16500 / 25000
17000 / 25000
17500 / 25000
18000 / 25000
18500 / 25000
19000 / 25000
19500 / 25000
20000 / 25000
20500 / 25000
21000 / 25000
21500 / 25000
22000 / 25000
22500 / 25000
23000 / 25000
23500 / 25000
24000 / 25000
24500 / 25000


In [11]:
model_fin = Sequential()
model_fin.add(Dense(7, activation = 'relu', input_shape = (3,)))
model_fin.add(Dense(20, activation = 'relu'))
model_fin.add(Dense(1, activation = 'sigmoid'))
model_fin.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model_fin.fit(final_train, y_train, epochs=7, batch_size=64)

Epoch 1/7
391/391 [==============================] - 1s 1ms/step - loss: 0.4955 - accuracy: 0.7700
Epoch 2/7
391/391 [==============================] - 0s 1ms/step - loss: 0.1461 - accuracy: 0.9486
Epoch 3/7
391/391 [==============================] - 0s 1ms/step - loss: 0.1393 - accuracy: 0.9497
Epoch 4/7
391/391 [==============================] - 0s 1ms/step - loss: 0.1422 - accuracy: 0.9490
Epoch 5/7
391/391 [==============================] - 0s 1ms/step - loss: 0.1367 - accuracy: 0.9514
Epoch 6/7
391/391 [==============================] - 0s 1ms/step - loss: 0.1328 - accuracy: 0.9518
Epoch 7/7
391/391 [==============================] - 0s 1ms/step - loss: 0.1414 - accuracy: 0.9492
